In [1]:
import pickle
import requests
import json
import pandas as pd

# Methods

Extract info given a match info. Methods used can be seen used under "Getting Dota Info P1" for more information

In [2]:
def count_chat(dota_dict):
    dire_talk, radiant_talk = 0,0
    for item in dota_dict['chat']:
        try:
            if item['player_slot'] > 127:
                dire_talk +=1
            else:
                radiant_talk+=1
        except:
            continue
    return dire_talk, radiant_talk
def get_hero_picked(lst):
    lst = dota_dict['draft_timings']
    radiant_hero = []
    dire_hero = []
    for item in lst:
        if item['pick'] == True:
            if item['player_slot'] > 4:
                dire_hero.append(item['hero_id'])
            else:
                radiant_hero.append(item['hero_id'])
    return radiant_hero, dire_hero
def get_courier_kills(dota_dict):
    radiant_kill_courier, dire_kill_courier, radiant_kill_rs, dire_kill_rs, radiant_aegis, dire_aegis = 0,0,0,0,0,0
    for item in dota_dict['objectives']:
        if item['type'] == 'CHAT_MESSAGE_COURIER_LOST':
            if item['killer'] > 127:
                dire_kill_courier+=1
            else:
                radiant_kill_courier +=1
        elif item['type'] == 'CHAT_MESSAGE_ROSHAN_KILL':
            if item['team'] ==3:
                dire_kill_rs +=1
            else:
                radiant_kill_rs+=1
        elif item['type'] == 'CHAT_MESSAGE_AEGIS':
            if item['player_slot'] > 127:
                dire_aegis+=1
            else:
                radiant_aegis +=1
    return radiant_kill_courier, dire_kill_courier,radiant_kill_rs, dire_kill_rs, radiant_aegis, dire_aegis
def get_gold_xp_adv(dota_dict):
    start = 11 #get to 10min
    if dota_dict['duration'] < 1200:
        start = dota_dict['duration']//60//2
    return sum(dota_dict['radiant_gold_adv'][1:start])/len(dota_dict['radiant_gold_adv'][1:start]) , sum(dota_dict['radiant_xp_adv'][1:start])/len(dota_dict['radiant_xp_adv'][1:start])
def get_player_infos(dota_dict):
    consumables =['tango','enchanted_mango','flask', 'clarity', 'farie_fire', 'dust', 'blood_grenade', 'smoke_of_deceit']#addfariefire and blood grenade and smoke dust
    ability_uses = ['ability_pluck_famango', 'twin_gate_portal_warp', 'ability_lamp_use']
    radiant_first_blood = None
    radiant_denies, radiant_runes,radiant_cosmetics, radiant_camps_stacked, radiant_last_hits, radiant_consumables, radiant_ability_used, radiant_obs,radiant_portal_used, radiant_fruit_plucked, radiant_lamp_used ,radiant_sent, dire_denies, dire_runes,dire_cosmetics, dire_camps_stacked, dire_last_hits, dire_consumables, dire_ability_used,dire_obs,dire_sent, dire_portal_used, dire_fruit_plucked, dire_lamp_used = 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
    for player in dota_dict['players']:
        if player['player_slot']<=127:
            if player['firstblood_claimed'] == 1:
                radiant_first_blood = True
            radiant_denies+= player['denies']
            radiant_last_hits+= player['last_hits']
            radiant_runes+= player['rune_pickups']
            radiant_cosmetics += len(player['cosmetics'])
            radiant_camps_stacked += player['camps_stacked']
            for key,pair in player['ability_uses'].items():
                if key not in ability_uses:
                    radiant_ability_used += pair
                elif key == 'ability_pluck_famango':
                    radiant_fruit_plucked += pair
                elif key == "twin_gate_portal_warp":
                    radiant_portal_used += pair
                elif key =="ability_lamp_use":
                    radiant_lamp_used += pair
            for key,pair in player['purchase'].items():
                if key in consumables:
                    radiant_consumables += pair
                elif key == 'ward_observer':
                    radiant_obs+= pair
                elif key=='ward_sentry':
                    radiant_sent += pair
        else:
            if player['firstblood_claimed'] == 1:
                radiant_first_blood = False
            dire_denies+= player['denies']
            dire_last_hits+= player['last_hits']
            dire_runes+= player['rune_pickups']
            dire_cosmetics += len(player['cosmetics'])
            dire_camps_stacked += player['camps_stacked']
            for key,pair in player['ability_uses'].items():
                if key not in ability_uses:
                    dire_ability_used += pair
                elif key == 'ability_pluck_famango':
                    dire_fruit_plucked += pair
                elif key == "twin_gate_portal_warp":
                    dire_portal_used += pair
                elif key =="ability_lamp_use":
                    dire_lamp_used += pair
            for key,pair in player['purchase'].items():
                if key in consumables:
                    dire_consumables += pair
                elif key == 'ward_observer':
                    dire_obs+= pair
                elif key=='ward_sentry':
                    dire_sent += pair


    return radiant_denies, radiant_runes,radiant_cosmetics, radiant_camps_stacked, radiant_last_hits, radiant_consumables, radiant_ability_used, radiant_obs,radiant_portal_used, radiant_fruit_plucked, radiant_lamp_used ,radiant_sent, dire_denies, dire_runes,dire_cosmetics, dire_camps_stacked, dire_last_hits, dire_consumables, dire_ability_used,dire_obs,dire_sent, dire_portal_used, dire_fruit_plucked, dire_lamp_used
def get_all_info(dota_dict):
    radiant_denies, radiant_runes,radiant_cosmetics, radiant_camps_stacked, radiant_last_hits, radiant_consumables, radiant_ability_used, radiant_obs,radiant_portal_used, radiant_fruit_plucked, radiant_lamp_used ,radiant_sent, dire_denies, dire_runes,dire_cosmetics, dire_camps_stacked, dire_last_hits, dire_consumables, dire_ability_used,dire_obs,dire_sent, dire_portal_used, dire_fruit_plucked, dire_lamp_used = get_player_infos(dota_dict)
    avg_gold_adv_early, avg_xp_adv_early = get_gold_xp_adv(dota_dict)
    radiant_kill_courier, dire_kill_courier,radiant_kill_rs, dire_kill_rs, radiant_aegis, dire_aegis= get_courier_kills(dota_dict)
    radiant_chat, dire_chat = count_chat(dota_dict)
    match_info = {
        "match_id": dota_dict['match_id'],
        "duration": dota_dict["duration"],
        "radiant_win": dota_dict["radiant_win"],
        "radiant_denies": radiant_denies,
        "radiant_runes": radiant_runes,
        "radiant_cosmetics": radiant_cosmetics,
        "radiant_camps_stacked": radiant_camps_stacked,
        "radiant_last_hits": radiant_last_hits,
        "radiant_consumables": radiant_consumables,
        "radiant_ability_used": radiant_ability_used,
        "radiant_obs": radiant_obs,
        "radiant_portal_used": radiant_portal_used,
        "radiant_fruit_plucked": radiant_fruit_plucked,
        "radiant_lamp_used": radiant_lamp_used,
        "radiant_sent": radiant_sent,
        "dire_denies": dire_denies,
        "dire_runes": dire_runes,
        "dire_cosmetics": dire_cosmetics,
        "dire_camps_stacked": dire_camps_stacked,
        "dire_last_hits": dire_last_hits,
        "dire_consumables": dire_consumables,
        "dire_ability_used": dire_ability_used,
        "dire_obs": dire_obs,
        "dire_sent": dire_sent,
        "dire_portal_used": dire_portal_used,
        "dire_fruit_plucked": dire_fruit_plucked,
        "dire_lamp_used": dire_lamp_used,
        "avg_gold_adv_early": avg_gold_adv_early,
        "avg_xp_adv_early": avg_xp_adv_early,
        "radiant_kill_courier": radiant_kill_courier,
        "dire_kill_courier": dire_kill_courier,
        "radiant_kill_rs": radiant_kill_rs,
        "dire_kill_rs": dire_kill_rs,
        "radiant_aegis": radiant_aegis,
        "dire_aegis": dire_aegis,
        "radiant_chat": radiant_chat,
        "dire_chat": dire_chat
    }
    return match_info

# Start

Load match ids from those in "Getting Dota Info P1"

In [3]:
with open("match_ids", "rb") as fp:
    match = pickle.load(fp)

In [4]:
len(match)

11801

In [5]:
type(match)

list

## First cleaning

Multiple cleanings are done. I believe this is because response may come out as promises without the actual value and hence the try and except to deal with them. (Can be improved)

In [9]:
matches = []
for match_id in match:
    try:
        print(match_id)
        response = requests.get(f"https://api.opendota.com/api/matches/{match_id}")
        match_data = response.json()
        match_info = get_all_info(match_data)
        matches.append(match_info)
    except:
        continue
df = pd.DataFrame(matches)

7115636736
7099875329
7237632006
7230619656
7221346319
7215906831
7099351056
7120060434
7180058643
7050133521
7073562645
7041843215
7199653917
7191232544
7175897120
7078477856
7035158560
7168753711
7046266928
7115931702
7217184825
7160954937
7122812986
7051477052
7192477757
7040204874
7134773328
7171670097
7126548563
7096762457
7131889760
7167246434
7087194210
7191429220
7126057061
7227441265
7046004853
7046103159
7150370936
7100596344
7164887164
7111016576
7221772418
7089389700
7144734855
7044530312
7230881931
7085588619
7101022350
7079428238
7079231635
7222984853
7113080983
7132446872
7095845015
7102759071
7050002591
7049805988
7224623274
7125860523
7145849004
7169966253
7074447535
7150403761
7046070449
7159513267
7039320250
7153287355
7091028158
7237238975
7228883135
7076413636
7175045318
7189659848
7095582924
7164592333
7136772304
7070744787
7047053528
7226425561
7152926937
7065665756
7201915104
7198212320
7167312102
7057440999
7151943916
7139754224
7149158642
7230423288
7141687549

7164921871
7150471189
7049250842
7058032667
7097288732
7171606557
7219677215
7195035679
7046563872
7134218283
7218333744
7054133299
7147685941
7098009655
7117015096
7110985785
7162366016
7167707205
7233636422
7169017926
7228164170
7195330635
7210469452
7125633102
7181305938
7165675606
7205947480
7228786779
7181305948
7104825436
7129204830
7083231329
7110330468
7141886060
7084771437
7214303342
7207979119
7064291441
7044860017
7193593971
7042533493
7210502265
7215941757
7186548862
7189071999
7109118079
7221905539
7218432140
7051675790
7060097169
7103416468
7221741719
7090178207
7220562084
7235700901
7205128356
7112362151
7054690468
7107381422
7208700081
7184058555
7057017024
7123798209
7082838211
7205783748
7034112196
7113738441
7034964169
7132186827
7054559440
7234816211
7102236884
7166560470
7156599001
7224232154
7132023002
7101384922
7144245476
7228885221
7042238701
7103908079
7129696496
7133595889
7057049839
7234357491
7055542512
7149095157
7086278903
7158335736
7087393019
7198345469

7173378011
7071535067
7106203613
7054757853
7189467112
7072747497
7176654827
7125077995
7080808428
7070257133
7186976751
7170035695
7168233455
7091425263
7111348211
7084707824
7115378677
7090376693
7061901298
7040176115
7078776825
7106170878
7056986111
7062786049
7102074883
7070486535
7234162699
7219777547
7083397131
7141232655
7171182610
7102009370
7235014685
7166660640
7079301155
7089000484
7106498597
7193006119
7229116456
7230296105
7170822188
7157977133
7223119919
7169511474
7141396530
7143755830
7225905210
7200772154
7143428158
7085396035
7133761609
7146344525
7188353102
7189270610
7221710931
7088148565
7172624473
7173476442
7220498523
7073927260
7159648352
7157157985
7041323110
7219253352
7122817128
7220760683
7041323117
7169773678
7077990510
7184715889
7064785009
7039684723
7099584632
7143067772
7232819326
7143198847
7069077637
7090278536
7099093129
7103451280
7173312658
7192744084
7179505813
7078842516
7184388247
7220465818
7230820509
7208865950
7072845983
7199428768
7218466977

7048206365
7209326625
7200839714
7153621027
7066261540
7212242981
7238031398
7124555814
7225186350
7111809070
7111284783
7161223218
7105091635
7084185653
7181703225
7111579706
7197300795
7049844806
7033395270
7234263112
7205492809
7194810442
7204247627
7168137290
7155980362
7094835278
7169579089
7153358930
7105943635
7053383766
7149066327
7171119193
7059380314
7134156892
7047190624
7080188006
7091886183
7106140270
7207983217
7063771250
7169579124
7065901175
7141103738
7091198074
7066261627
7220828286
7044864127
7159158919
7048140936
7165745297
7148279954
7198742675
7233116308
7050827921
7115315354
7095359645
7223810211
7176394923
7104993454
7158438081
7053777089
7202314437
7159355589
7041784009
7203952843
7079303378
7198841043
7048239314
7046830295
7165057240
7037065432
7223187676
7072946397
7159716062
7187863779
7141759205
7212832998
7177902312
7136057578
7224662251
7199594731
7191173355
7185766637
7088052470
7229741306
7205624060
7046699263
7074420997
7137007881
7060494603
7156865298

7090380824
7189733409
7112433698
7209951272
7069278250
7195467820
7198351405
7119118380
7073275950
7082287152
7033036844
7061086260
7161978937
7125835834
7151984703
7156473920
7090217024
7213785161
7141597260
7124492364
7138418770
7204249684
7209754716
7136649312
7194026082
7127212131
7189635173
7238393958
7043063909
7033462885
7103357038
7195697264
7043981424
7186817141
7073734784
7058628736
7124361346
7095787652
7076847749
7183736966
7220371596
7165157516
7052468370
7087595667
7205003412
7215456405
7193698453
7059710102
7070523545
7206543514
7055646878
7095492769
7068852387
7213686960
7068590259
7204839604
7098605748
7113842874
7045587137
7237148866
7236034754
7105552582
7103357128
7215128781
7183540429
7226269903
7091855568
7094411475
7201464532
7131832534
7235838170
7214833883
7093002467
7070490853
7150444777
7131406573
7070687474
7109910772
7124361476
7180820744
7075799304
7074652430
7070720271
7174627602
7146119443
7170990358
7201497367
7048175900
7166370081
7110500646
7152181545

...

7192304139
7158094348
7126768141
7100848654
7228742159
7123163670
7212128796
7233690145
7191091745
7178213922
7048157730
7034821161
7089904171
7097768492
7158520365
7081843247
7208426032
7204756016
7054776879
7188208181
7049861695
7084038730
7223433803
7224875596
7039113804
7034034771
7162550869
7217109590
7159306842
7200266844
7179459165
7158684253
7209310817
7168645738
7121328748
7099898476
7065721459
7172840053
7182244472
7188372092
7230085761
7214783107
7059888772
7234935436
7085021839
7082498710
7099832986
7084038811
7065524891
7053073055
7121132192
7108024996
7213701797
7095179941
7174118070
7175690935
7128636090
7092624058
7226120892
7130176189
7094491836
7086660286
7146756805
7089543877
7209376456
7078206157
7185259218
7169694418
7063722709
7037737688
7220550365
7233395422
7200496351
7057562336
7167630051
7083547363
7108713191
7126506219
7125949168
7102192368
7125457650
7128701684
7120935668
7171070712
7237065465
7062379257
7096031997
7162911486
7195384575
7089511166
7121689345

7174152761
7067558461
7086563902
7205577280
7167205956
7222747717
7220453964
7134208588
7102128718
7194501712
7058350673
7100719704
7167173209
7167369820
7171039837
7179395678
7050289760
7218061921
7122346594
7038165602
7112352356
7206363762
7193354866
7226155642
7093412474
7155147388
7054844543
7122969221
7162389126
7160554121
7221633676
7128408718
7052649106
7071228563
7224648342
7058449048
7230022298
7070442139
7090102940
7203381917
7121461919
7058940576
7149281953
7191945890
7103767202
7139582643
7161045684
7188472502
7069917889
7126508226
7068836549
7232938697
7087776459
7089775308
7038591695
7120282320
7065133786
7237165795
7225991910
7124148968
7076176616
7069557480
7189750507
7223272172
7100621547
7061201644
7057105644
7196893936
7078634227
7217898231
7132177144
7159309051
7097737979
7086629629
7040230145
7212229379
7143285510
7072801552
7150560020
7069655828
7111238424
7128802073
7223894810
7125590814
7045473056
7166518049
7076471587
7168058152
7106552618
7186146091
7075521328

7163013573
7046687175
7223994829
7195945421
7160916432
7144663505
7089842643
7034005971
7162227158
7068215766
7180872152
7194995163
7068051932
7090104798
7219079648
7170386401
7090072033
7235430883
7107537380
7104555493
7069198823
7173990888
7222094313
7123593705
7195191787
7216851436
7146007023
7056878067
7213574645
7135848955
7108159996
7150692861
7234710016
7158393347
7234021895
7092955657
7166257674
7100852752
7051766290
7134046739
7129295379
7237364249
7219341850
7191882267
7231433244
7069166114
7234578984
7044229674
7138110002
7040297523
7219046965
7127394870
7076014647
7165897272
7085353532
7045704252
7128083007
7187655239
7074933321
7066675787
7177988684
7105440338
7070542418
7107766875
7087286876
7066905179
7043279455
7186868834
7173007972
7100361316
7048096358
7225993831
7126903399
7088695913
7061400168
7198173803
7093545579
7193356912
7176383090
7090236019
7198567032
7164193405
7049046653
7122414208
7059630722
7076506243
7165307526
7036136071
7191653006
7221963411
7038986899

7101477207
7068414295
7207874908
7186379101
7095972196
7131296102
7170847080
7144665449
7071920491
7138242924
7042003313
7188246898
7202959731
7064121716
7223046520
7197159801
7176286584
7131033979
7125528953
7074738556
7217574271
7225569665
7197520263
7123923335
7109374344
7138832778
7229895052
7088271756
7151055248
7184675217
7147778449
7125856660
7195455896
7182971290
7164948895
7126512031
7141716385
7095841184
7131165091
7235826084
7078801825
7203418535
7192015276
7170126252
7148859820
7134441901
7037579698
7134310836
7218196917
7156265398
7043641783
7211643320
7155642812
7208923582
7181169087
7127527870
7237988811
7089942988
7076344268
7194014163
7211774420
7069659608
7136702941
7188312551
7064121832
7175533035
7128805869
7198208494
7211872765
7152726527
7213609475
7037383174
7188083215
7131918863
7033156111
7197520403
7084995091
7052358167
7074542105
7206433310
7182119454
7163343399
7101018664
7094137386
7201583660
7070642738
7098823223
7202992698
7083455034
7148204607
7035286081

7217936734
7202044255
7118387554
7199684963
7232780645
7110031722
7236483435
7178353007
7168686447
7206631794
7100004723
7192541557
7036664184
7126120830
7059700094
7216298368
7173568896
7141521796
7214823813
7166589318
7231961479
7183759750
7221705097
7092894087
7118748043
7074085260
7050951045
7207418255
7130249616
7149025681
7176059289
7214889389
7149222318
7193229743
7157643702
7110621626
7054195135
7092304320
7107443140
7119305161
7069104590
7184742865
7212497364
7224523224
7035976153
7221672412
7059601890
7202798052
7194147304
7130839529
7088142825
7199881709
7100135918
7122123248
7119501810
7213087224
7230388730
7052294655
7035484679
7138441739
7122975251
7226849813
7224949270
7082801687
7073069597
7226522143
7094008353
7088732706
7092730404
7175895592
7187397174
7150402114
7041776194
7073004100
7074118214
7235762760
7189461583
7143520855
7217183321
7155907164
7089060446
7068023397
7099546214
7120157287
7218199151
7185463927
7207352952
7131069049
7062485623
7188937341
7129627261

## Saving data

In [ ]:
df

In [10]:
df.to_csv('dota.csv')

# Open dataset

Open current data from first cleaning into a set. Then re-run the remaining matches with get_all_info

In [6]:
curr_data = pd.read_csv('dota.csv')

In [13]:
curr_set = set(curr_data['match_id'])

In [14]:
curr_set

{7115636736,
 7099875329,
 7237632006,
 7230619656,
 7221346319,
 7215906831,
 7099351056,
 7120060434,
 7050133521,
 7041843215,
 7073562645,
 7199653917,
 7191232544,
 7175897120,
 7078477856,
 7035158560,
 7168753711,
 7046266928,
 7115931702,
 7217184825,
 7160954937,
 7122812986,
 7051477052,
 7192477757,
 7040204874,
 7134773328,
 7171670097,
 7126548563,
 7096762457,
 7131889760,
 7167246434,
 7087194210,
 7126057061,
 7227441265,
 7046004853,
 7046103159,
 7100596344,
 7150370936,
 7164887164,
 7111016576,
 7221772418,
 7089389700,
 7144734855,
 7044530312,
 7230881931,
 7085588619,
 7101022350,
 7079428238,
 7079231635,
 7222984853,
 7113080983,
 7132446872,
 7095845015,
 7102759071,
 7050002591,
 7049805988,
 7224623274,
 7125860523,
 7095910693,
 7074578729,
 7151976748,
 7041777968,
 7038173490,
 7111737651,
 7071891763,
 7176487229,
 7120224573,
 7219446080,
 7121338689,
 7076315458,
 7081132356,
 7195722054,
 7128285512,
 7045349711,
 7223542096,
 7159546200,
 7129530712,

## Second cleaning

In [17]:
matches1 = []
for match_id in match:
    if match_id not in curr_set:
        try: 
            print(match_id)
            response = requests.get(f"https://api.opendota.com/api/matches/{match_id}")
            match_data = response.json()
            match_info = get_all_info(match_data)
            matches1.append(match_info)
        except:
            continue


7180058643
7191429220
7145849004
7169966253
7074447535
7150403761
7046070449
7159513267
7039320250
7153287355
7091028158
7237238975
7228883135
7076413636
7175045318
7189659848
7095582924
7164592333
7136772304
7070744787
7047053528
7226425561
7152926937
7065665756
7201915104
7198212320
7167312102
7057440999
7151943916
7139754224
7149158642
7230423288
7141687549
7079723265
7211843843
7162593542
7238385928
7079231755
7093321999
7186710803
7230914837
7138312469
7115964693
7222067488
7177404707
7163150628
7144440101
7037551056
7200834004
7100563925
7089619414
7089390046
7123075551
7230489058
7196836322
7112524258
7048298982
7203914215
7207191017
7212859882
7223149035
7139688938
7201718765
7227507183
7169573360
7234585077
7200801272
7083852281
7184712188
7085326855
7070450187
7091552782
7210697231
7196148239
7213777426
7137034774
7067402775
7134413342
7153779231
7057343010
7068811813
7068090918
7168852519
7090340391
7198507568
7216562738
7095943730
7224492598
7195918903
7171605046
7208010297

7084281616
7134940946
7223447315
7200542484
7042338578
7147851542
7068127000
7143296793
7212797729
7138873122
7087460129
7122259749
7186190121
7085526829
7083888431
7062458160
7208341298
7185141556
7205326646
7156567868
7034703678
7076089663
7081168704
7217975107
7046696772
7075893062
7175245640
7131270990
7082774356
7196512086
7160565594
7207784284
7186419551
7198904161
7210602340
7227510630
7094374252
7174721391
7158534000
7122521967
7163613042
7058296694
7191302014
7169019778
7067471749
7190908809
7117082512
7046336400
7115673491
7057608595
7164235672
7219122082
7200214946
7040536486
7146246056
7233048490
7169118122
7061966777
7143755830
7225905210
7200772154
7143428158
7085396035
7133761609
7146344525
7188353102
7189270610
7221710931
7088148565
7172624473
7173476442
7220498523
7073927260
7159648352
7157157985
7041323110
7219253352
7122817128
7220760683
7041323117
7169773678
7077990510
7184715889
7064785009
7039684723
7099584632
7143067772
7232819326
7143198847
7069077637
7090278536

7185210699
7036116302
7034674510
7178165589
7128915285
7189732698
7223254368
7057448293
7231905128
7071866216
7070489962
7101586795
7032872300
7228595565
7197269361
7194778993
7198842227
7111155058
7148707189
7144021365
7156669816
7186980218
7089003900
7175708029
7117348221
7036050815
7173709184
7096900992
7199923587
7228956039
7185735048
7160995208
7087037834
7147101580
7200873869
7166631310
7156768147
7048076700
7042702749
7084744115
7166729681
7064395256
7098441216
7102471746
7050829379
7071473221
7202676299
7071243853
7237148238
7189274190
7101914704
7169744465
7130783314
7081172558
7212375637
7154114133
7072390741
7178919514
7167975002
7165419163
7159062184
7062331113
7179345657
7128325881
7078715130
7215554302
7181016830
7218405120
7055646466
7163715332
7101619973
7218470667
7171251983
7075471119
7156768533
7156604694
7089135384
7185407775
7145168679
7044144943
7088938802
7196024628
7087234869
7182032694
7158112056
7093952315
7073636158
7176593219
7184850757
7067672390
7208968008

7160310805
7226108950
7176629276
7103392818
7233776786
7101131954
7110700225
7126920393
7153069260
7036054733
7182822606
7201860817
7098313941
7224765654
7238266071
7159688406
7081929946
7051095261
7125249249
7094512865
7067970792
7128263914
7044050157
7173221614
7033629937
7138389235
7162539253
7175974137
7131245817
7036185849
7177776382
7149071615
7082552577
7221816584
7164177673
7083044108
7044279566
7086976272
7106931987
7082945814
7123152152
7205498139
7164144924
7234104606
7217687840
7158607137
7215852836
7058959652
7075310889
7180725549
7140224303
7209135408
7147597105
7186066738
7171747122
7195372856
7093497150
7078554942
7171419460
7134522694
7220374855
7057878342
7079963977
7229648205
7161064786
7135931733
7227387224
7097724249
7054142825
7166176618
7167028593
7110045042
7115681139
7215656308
7176793541
7185083966
7219293759
7199632959
7101034046
7174073924
7107194447
7082061392
7203958356
7062695510
7108603480
7217753690
7131999836
7107391072
7145238118
7057485415
7222570601

7167131185
7118503468
7068892718
7048216116
7038778925
7152254518
7058505271
7196982841
7180435002
7129480762
7182925373
7099989566
7158316607
7093632576
7100382785
7192395330
7088225853
7199899209
7059848783
7067549264
7073480277
7162838615
7046413911
7036190295
7196294747
7171063389
7133118045
7162084959
7091240544
7038811745
7059488354
7068270179
7037468254
7089012326
7071907431
7161724521
7227063915
7106379372
7044972139
7070662255
7183908466
7128727157
7103168118
7175487095
7056998012
7238270590
7207599752
7189610120
7147601545
7051689609
7161101965
7114145423
7103201065
7217495858
7140425528
7090618169
7167524666
7039795000
7087505214
7209500481
7170146114
7158677314
7107854147
7180173125
7211401033
7185121099
7090159440
7076888410
7133970269
7101595486
7098351457
7075643235
7202324325
7230898029
7167131502
7071088496
7103922037
7121256314
7215038339
7227162500
7211925380
7037501319
7048347529
7044120457
7107624844
7035502481
7061159831
7218479007
7119126431
7210516386
7235583908

7059393993
7168314826
7168216524
7106907597
7032982988
7226838482
7168937428
7085575701
7166610966
7120965232
7197642359
7214222968
7124471427
7196102277
7189548678
7182241414
7083544200
7166381707
7057329806
7175982739
7126240927
7140232870
7089901225
7229198002
7217467058
7204097715
7111233207
7198166712
7164710584
7176900290
7151898309
7051562701
7213928143
7195709136
7061032658
7162810070
7234277079
7203638999
7120473818
7059197659
7050383070
7074270944
7152389858
7074402023
7033147111
7199510261
7152520951
7168282361
7154192121
7104876283
7143902973
7200100098
7063359237
7143116550
7205310218
7209176846
7124078353
7097896725
7218810646
7142330138
7114542879
7234801440
7202721571
7070404391
7058083624
7159435050
7237193516
7112445742
7147605807
7108808498
7130468155
7033868094
7158878018
7230934856
7096684364
7057559374
7209832302
7174180723
7206260619
7164219277
7161139105
7195971545
7150456828
7151407107
7206129669
7134007307
7196332045
7218581518
7178997778
7035441171
7180013594

7099703010
7236738789
7237000934
7103569640
7060741867
7183916782
7186669295
7067295477
7155146490
7183916798
7162650367
7117102847
7048093439
7209836301
7238344462
7073423127
7199645467
7119331101
7082860326
7206592298
7104847659
7236968236
7112810287
7122476857
7102914378
7091216242
7168843646
7125327787
7051436008
7227432938
7127293940
7069032439
7130537977
7219011579
7175036924
7223500797
7155539966
7126048768
7162683393
7119101952
7154622470
7056875528
7120805900
7091740685
7148953617
7228481554
7048355858
7075880981
7070408727
7221927960
7091085337
7143645211
7196106783
7126704159
7057170466
7208427558
7219863592
7073390635
7095771180
7098032174
7182311473
7074996276
7139516470
7097737272
7149412410
7087120444
7165141059
7170613317
7235756102
7166156870
7121068109
7219273806
7167533134
7138369616
7071588434
7155114068
7061233752
7199711324
7205249123
7119036519
7104520296
7115300970
7086760044
7130210413
7191453808
7100850298
7208034427
7154917500
7159079039
7215341696
7219404937

7138274498
7108521153
7219539148
7205874892
7073426636
7046622415
7205055697
7087877329
7063366866
7215869140
7072574678
7108586711
7072050392
7164587231
7144041695
7033875680
7073328354
7195520227
7126052070
7191031017
7062777073
7125495027
7212756212
7141027061
7162948856
7167601915
7145942273
7057173763
7206399237
7238479113
7174483211
7147449612
7080865068
7199976757
7120022865
7175695711
7157443954
7124610471
7088860586
7202270636
7183855023
7236414898
7180316083
7078374834
7133883830
7127362998
7033646518
7142763963
7077588413
7137750465
7221276103
7107833287
7115402697
7036300759
7089614296
7121006041
7036104151
7159672284
7065103838
7090269663
7207841249
7213051362
7223307748
7204924900
7174024677
7129361893
7095447013
7087844840
7232613866
7057305063
7153446385
7216164344
7185722872
7111142906
7179038211
7204105739
7195717131
7154265615
7128673809
7106293270
7191359001
7184215579
7197158940
7153446429
7149121051
7097085467
7069724189
7033876000
7226256930
7213837859
7215115813

7212039035
7179074431
7117011841
7075167106
7093156739
7196375944
7167802250
7089159050
7075691405
7094664083
7175601044
7217281943
7209122714
7040400285
7215348640
7222262689
7183694762
7238253487
7196703667
7136869299
7060913076
7228947384
7213972408
7197490104
7088470969
7214300096
7182351298
7226915783
7217642439
7139883981
7079820245
7176059865
7198178266
7036730331
7107148787
7038860283
7168556067
7045905443
7070710825
7107181645
7128185981
7141620865
7098530947
7205911690
7139130509
7177534609
7169801364
7094008980
7176584342
7170555033
7074086043
7062650013
7183269023
7099710624
7071366306
7212006564
7033519270
7128022183
7164132528
7155874993
7157873842
7069662389
7189626040
7100333244
7237926077
7100398785
7180713156
7219150021
7165213893
7169735879
7154334916
7047773387
7101349069
7162887375
7221673173
7159676119
7146077400
7172979929
7117372632
7090142424
7099546846
7115275487
7199030496
7113800930
7191624935
7102233834
7124647148
7071235312
7051050227
7069334772
7092698357

In [20]:
df1 = pd.DataFrame(matches1)

In [22]:
result = pd.concat([curr_data,df1])

In [26]:
len(result['match_id'].unique())

9047

In [19]:
len(matches1)

3472

## Last cleaning

In [28]:
curr_set = set(result['match_id'])

In [29]:
remaining_match = [match_id for match_id in match if match_id not in curr_set]

In [31]:
len(remaining_match)

2754

In [32]:
matches2 = []
for match_id in remaining_match:
    try:
        print(match_id)
        response = requests.get(f"https://api.opendota.com/api/matches/{match_id}")
        match_data = response.json()
        match_info = get_all_info(match_data)
        matches2.append(match_info)
    except:
        continue

7169573360
7189496380
7115375164
7233077837
7142277710
7099318863
7070876242
7096992339
7203390036
7191855701
7102038614
7070286419
7233405529
7159611994
7087030878
7041614431
7165772386
7169278566
7211385452
7177929324
7067140717
7142670962
7056982642
7107412596
7089586804
7101022838
7090897528
7201882756
7088046752
7189103284
7233700574
7070647062
7072482071
7131169560
7203128089
7168262937
7122060059
7153320736
7226589985
7238386466
7036371747
7038436132
7099843373
7169147694
7144833838
7214105393
7099024177
7210435382
7173178176
7068648260
7231210309
7082115910
7179895623
7081427783
7059637065
7068713807
7097648315
7173833916
7089292481
7071532225
7164822724
7058490570
7142081748
7135659220
7237207254
7175341270
7155582173
7134774494
7079199967
7074481375
7075005667
7219086566
7062291692
7129400559
7048135921
7175341299
7057900789
7052395771
7056327933
7103022365
7065372220
7210731069
7065830973
7182059071
7121602112
7167444545
7206307394
7186810436
7109445191
7080904267
7063570000

7223190566
7196615728
7193142320
7185048627
7056892989
7221945423
7118627962
7210378401
7186719990
7178593817
7197402650
7214310940
7076390431
7102866977
7058204196
7183541799
7231546921
7196747306
7059645994
7228958253
7113025074
7186916915
7069214260
7046211126
7207691832
7207560762
7211591284
7061612204
7110141614
7141009142
7199139583
7037626113
7182296840
7130294029
7206840085
7176431381
7231514394
7156770586
7130621723
7232202529
7109158690
7156213542
7040378668
7187572530
7211624243
7066855221
7129311030
7204316991
7220537153
7115122497
7070820164
7143630662
7040280392
7167420236
7049783120
7093233489
7214966613
7079208790
7077242712
7071573852
7214147421
7237379934
7106766687
7070000993
7100933988
7043721069
7071344494
7199008625
7188195187
7144613748
7131146102
7074981751
7082518392
7199434619
7046604822
7045589017
7183804446
7057025055
7041820708
7070197800
7054895144
7226894383
7143270447
7100835887
7083763762
7060072499
7036217397
7066363958
7121578039
7134324795
7068592187

7198655065
7113392730
7194952283
7183483483
7215104608
7173751392
7158350435
7067222627
7223493221
7165788782
7085408879
7040287343
7136068210
7183483511
7226114682
7043564154
7217726076
7048348284
7194592070
7122404291
7088587716
7092290504
7094223820
7217824722
7124698067
7100187602
7123059679
7139804129
7213597666
7156384740
7203668971
7128990704
7112377330
7128761338
7213711359
7129089019
7108477953
7218217986
7217398788
7196230661
7044285447
7206126600
7184892937
7050118152
7094453259
7206650897
7079871505
7139542037
7196492827
7182042141
7046349856
7162872866
7231980580
7090914342
7072990247
7068304431
7169688652
7175849097
7199868045
7187514531
7070958799
7168312529
7068009999
7169558035
7169525268
7176570391
7181583900
7067125277
7238501921
7039927843
7189055015
7046776360
7069320754
7231391283
7156909620
7237027381
7048939063
7065716280
7219234363
7064045118
7163627072
7158253121
7203767882
7112934990
7064831993
7060703224
7171491847
7083051045
7147702321
7143376946
7177914420

7140174137
7034759486
7122413888
7120906564
7068313925
7081158983
7230679375
7179659603
7125461337
7067986270
7061039457
7088662885
7197157734
7184181606
7167175021
7143188852
7070771576
7155476857
7094495612
7040133501
7169730942
7151708550
7080995208
7214885259
7194372491
7216753037
7169862028
7129360790
7087221151
7144597920
7193356705
7171565985
7133718961
7093086642
7049439665
7047834033
7141222839
7097805243
7169763777
7198009794
7169370564
7220062661
7163013573
7046687175
7223994829
7195945421
7160916432
7144663505
7089842643
7034005971
7162227158
7068215766
7180872152
7194995163
7068051932
7090104798
7219079648
7170386401
7090072033
7235430883
7107537380
7104555493
7069198823
7173990888
7222094313
7053929137
7124610471
7197158940
7153446429
7149121051
7097085467
7069724189
7033876000
7226256930
7213837859
7215115813
7168978473
7202237997
7145582126
7046557233
7117336146
7172714111
7063957145
7126118055
7206661832
7103475403
7070019287
7230254811
7115337441
7109471982
7149088497

In [34]:
len(matches2)
df2 = pd.DataFrame(matches2)
final_result = pd.concat([result,df2])

In [45]:
final_result = final_result.drop(['Unnamed: 0'], axis=1)

In [48]:
final_result

,match_id,duration,radiant_win,radiant_denies,radiant_runes,radiant_cosmetics,radiant_camps_stacked,radiant_last_hits,radiant_consumables,radiant_ability_used,...,avg_gold_adv_early,avg_xp_adv_early,radiant_kill_courier,dire_kill_courier,radiant_kill_rs,dire_kill_rs,radiant_aegis,dire_aegis,radiant_chat,dire_chat
0,7115636736,2347,False,55,14,18,9,759,33,597,...,246.5,289.6,1,1,0,1,0,1,28,19
1,7099875329,1383,False,5,1,5,2,304,16,402,...,-4111.9,-1667.3,0,3,0,1,0,1,23,21
2,7237632006,2117,False,48,24,6,12,865,32,1019,...,-345.7,-463.3,1,2,0,2,0,1,66,35
3,7230619656,1865,True,35,18,4,8,709,26,594,...,-647.1,-213.7,1,2,0,0,0,0,18,3
4,7221346319,1656,False,42,9,3,5,591,21,533,...,55.3,190.5,2,1,0,0,0,0,13,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1468,7094992880,2288,False,22,21,2,4,767,46,774,...,-1475.7,-1691.7,1,4,1,1,1,1,65,32
1469,7181697009,2132,True,59,24,10,16,1043,29,521,...,1712.5,1135.5,0,0,2,0,2,0,32,34
1470,7071825904,1803,True,44,21,10,7,718,35,529,...,463.3,618.0,3,0,1,0,1,0,47,18
1471,7037714420,1684,True,26,12,15,2,646,20,541,...,-1457.3,-1288.0,0,0,1,0,1,0,26,9


In [50]:
final_result = final_result.reset_index(drop=True)

In [53]:
final_result

,match_id,duration,radiant_win,radiant_denies,radiant_runes,radiant_cosmetics,radiant_camps_stacked,radiant_last_hits,radiant_consumables,radiant_ability_used,...,avg_gold_adv_early,avg_xp_adv_early,radiant_kill_courier,dire_kill_courier,radiant_kill_rs,dire_kill_rs,radiant_aegis,dire_aegis,radiant_chat,dire_chat
0,7115636736,2347,False,55,14,18,9,759,33,597,...,246.5,289.6,1,1,0,1,0,1,28,19
1,7099875329,1383,False,5,1,5,2,304,16,402,...,-4111.9,-1667.3,0,3,0,1,0,1,23,21
2,7237632006,2117,False,48,24,6,12,865,32,1019,...,-345.7,-463.3,1,2,0,2,0,1,66,35
3,7230619656,1865,True,35,18,4,8,709,26,594,...,-647.1,-213.7,1,2,0,0,0,0,18,3
4,7221346319,1656,False,42,9,3,5,591,21,533,...,55.3,190.5,2,1,0,0,0,0,13,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10515,7094992880,2288,False,22,21,2,4,767,46,774,...,-1475.7,-1691.7,1,4,1,1,1,1,65,32
10516,7181697009,2132,True,59,24,10,16,1043,29,521,...,1712.5,1135.5,0,0,2,0,2,0,32,34
10517,7071825904,1803,True,44,21,10,7,718,35,529,...,463.3,618.0,3,0,1,0,1,0,47,18
10518,7037714420,1684,True,26,12,15,2,646,20,541,...,-1457.3,-1288.0,0,0,1,0,1,0,26,9


## Save final dataframe to be used for analysis

In [55]:
final_result
final_result.to_csv('dota.csv', index=False)

In [16]:


response = requests.get(f"https://api.opendota.com/api/matches/7223149035")
match_data = response.json()
match_info = get_all_info(match_data)